In [ ]:
import numpy as np
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('stripped_storms.csv', header=None, skiprows = 1)
# 258 unique hurricanes
pd.set_option('display.max_rows', None)

print(df[0].value_counts())

0
Bonnie       273
Bertha       261
Dennis       244
Florence     239
Claudette    237
Emily        236
Alberto      233
Josephine    231
Helene       228
Edouard      228
Frances      215
Danielle     214
Earl         208
Felix        206
Lili         204
Arlene       199
Harvey       198
Chris        198
Danny        197
Gabrielle    196
Gordon       196
Ana          189
Erin         184
Beryl        181
Irene        176
Leslie       175
Ivan         175
Ophelia      175
Arthur       172
Gert         171
Chantal      171
Debby        170
Isaac        170
Isidore      168
Barry        165
Cindy        164
Karl         163
Charley      163
Maria        161
Gustav       161
Jeanne       158
Humberto     158
Henri        154
Ernesto      152
Jose         152
Dean         148
Lisa         144
Gloria       144
Jerry        138
Bret         138
Grace        135
Kate         133
Karen        132
Floyd        131
Kyle         127
Nadine       127
Alex         125
Erika        124
Otto        

In [ ]:
df = pd.read_csv('triplets.csv', header=None, skiprows = 1)

df = df.sample(frac = 1, random_state= 42)

data = []

# Convert df dataframe to list of lists
for index, row in df.iterrows():
    data.append(row.tolist())

# Convert list of lists to numpy array
data = np.array(data)

# Construct encoder, decoder input and target
X_encoder = []
X_decoder = []
y_decoder = []

# X_encode = everything in the row from 0 to row length - 6. Then, X_encode is separated into list of lists
for i in range(len(data)):
    X_encoder.append(data[i][:12])
    X_decoder.append(data[i][6:])
    y_decoder.append(list(data[i][9:12]) + list(data[i][15:18]))

# Convert lists to numpy arrays
X_encoder = np.array(X_encoder).reshape(len(X_encoder), 2, 6)
X_decoder = np.array(X_decoder).reshape(len(X_decoder), 2, 6)
y_decoder = np.array(y_decoder).reshape(len(X_decoder), 2, 3)

# split into training, validation, and data set

print("X_encoder shape:", X_encoder.shape)
print("X_decoder shape:", X_decoder.shape)
print("y_decoder shape:", y_decoder.shape)



X_encoder shape: (18550, 2, 6)
X_decoder shape: (18550, 2, 6)
y_decoder shape: (18550, 2, 3)


In [ ]:
from sklearn.model_selection import train_test_split

# Split into training and temp (validation + test) sets (60-40 split)
X_encoder_train, X_encoder_temp, X_decoder_train, X_decoder_temp, y_decoder_train, y_decoder_temp = \
    train_test_split(X_encoder, X_decoder, y_decoder, test_size=0.4, shuffle=False)

# Split temp into validation and test sets (50-50 split of the remaining 40%)
X_encoder_val, X_encoder_test, X_decoder_val, X_decoder_test, y_decoder_val, y_decoder_test = \
    train_test_split(X_encoder_temp, X_decoder_temp, y_decoder_temp, test_size=0.5, shuffle=False)

print("Training set shapes:")
print("X_encoder_train:", X_encoder_train.shape)
print("X_decoder_train:", X_decoder_train.shape)
print("y_decoder_train:", y_decoder_train.shape)

print("\nValidation set shapes:")
print("X_encoder_val:", X_encoder_val.shape)
print("X_decoder_val:", X_decoder_val.shape)
print("y_decoder_val:", y_decoder_val.shape)

print("\nTesting set shapes:")
print("X_encoder_test:", X_encoder_test.shape)
print("X_decoder_test:", X_decoder_test.shape)
print("y_decoder_test:", y_decoder_test.shape)

Training set shapes:
X_encoder_train: (11130, 2, 6)
X_decoder_train: (11130, 2, 6)
y_decoder_train: (11130, 2, 3)

Validation set shapes:
X_encoder_val: (3710, 2, 6)
X_decoder_val: (3710, 2, 6)
y_decoder_val: (3710, 2, 3)

Testing set shapes:
X_encoder_test: (3710, 2, 6)
X_decoder_test: (3710, 2, 6)
y_decoder_test: (3710, 2, 3)


In [ ]:
num_features = 6
time_step = 2
num_decoder_features = 3
LSTM_hidden_dim = 64
LSTM_cell_activation = 'tanh'
LSTM_gate_activation = 'sigmoid'
dense_activation = 'linear'
batch_size = 64
epochs = 100
optimizer = 'adam'
loss = 'mean_squared_error'

In [ ]:
import tensorflow as tf
from tensorflow.keras import backend as K

def percentage_deviation(y_true, y_pred):
    epsilon = K.epsilon()
    percentage_dev = tf.reduce_mean(tf.abs((y_true - y_pred) / (y_true + epsilon))) * 100
    return percentage_dev

In [ ]:
num_features = 6
time_step = 2
num_decoder_features = 3
LSTM_hidden_dim = 64
LSTM_cell_activation = 'tanh'
LSTM_gate_activation = 'sigmoid'
dense_activation = 'linear'
batch_size = 64
epochs = 50
optimizer = 'adam'
loss = 'mean_squared_error'
# Define the encoder
encoder_inputs = Input(shape=(None, num_features))
encoder_lstm = LSTM(LSTM_hidden_dim, return_state=True, activation = LSTM_cell_activation, recurrent_activation=LSTM_gate_activation)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c]

# Define the decoder
decoder_inputs = Input(shape=(None, num_features))
decoder_lstm = LSTM(LSTM_hidden_dim, return_sequences=True, return_state=True, activation = LSTM_cell_activation, recurrent_activation=LSTM_gate_activation)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_features, activation = dense_activation)
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile the model
model.compile(optimizer, loss, metrics = [percentage_deviation])

# Print the model summary
model.summary()

# explain the input and output shape of each of the layers.

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2             │ (None, None, 6)        │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_3             │ (None, None, 6)        │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_2 (LSTM)             │ [(None, 64), (None,    │         18,176 │ input_layer_2[0][0]    │
│                           │ 64), (None, 64)]       │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_3 (LSTM)             │ [(None, None, 64),     │         18,176 │ input_layer_3[0][0],   │
│                           │ (None, 64), (None,     │                │ lstm_2[0][1],          │
│                           │ 64)]                   │                │ lstm_2[0][2]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, None, 3)        │            195 │ lstm_3[0][0]           │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 36,547 (142.76 KB)

 Trainable params: 36,547 (142.76 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# data imputation model
# loss function compare against all 6 points
# more points per timestep (start with 10)
# combine month date hour into one feature
# look into TimeSeriesSplit
# try the original model


In [ ]:
# Train the model
model.fit([X_encoder_train, X_decoder_train], y_decoder_train, batch_size, epochs, validation_data=([X_encoder_val, X_decoder_val], y_decoder_val))

Epoch 1/50
174/174 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - loss: 2360.8167 - percentage_deviation: 336077.3750 - val_loss: 1552.6506 - val_percentage_deviation: 59.7886
Epoch 2/50
174/174 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 1384.3188 - percentage_deviation: 764483.4375 - val_loss: 1015.3453 - val_percentage_deviation: 42.8330
Epoch 3/50
174/174 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 914.3341 - percentage_deviation: 1891396.0000 - val_loss: 705.1210 - val_percentage_deviation: 36.9907
Epoch 4/50
174/174 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 644.3607 - percentage_deviation: 2615717.0000 - val_loss: 528.2434 - val_percentage_deviation: 33.5132
Epoch 5/50
174/174 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 498.2753 - percentage_deviation: 1551452.8750 - val_loss: 419.7078 - val_percentage_deviation: 31.2381
Epoch 6/50
174/174 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - loss: 395.2415 - percentage_deviation: 2325471.0000 - val_loss: 339.6247 - val_percentage_deviation: 28.7451
Epoch 7/50
174/17

In [ ]:
# training curve plotter

In [ ]:
# Define the encoder model for inference
encoder_inputs = model.input[0]  # Input of the encoder
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # LSTM layer outputs
encoder_states = [state_h_enc, state_c_enc]
encoder_model = Model(encoder_inputs, encoder_states)

# Define the decoder model for inference
decoder_inputs = model.input[1]  # Input of the decoder

# LSTM layer
decoder_state_input_h = Input(shape=(LSTM_hidden_dim,))
decoder_state_input_c = Input(shape=(LSTM_hidden_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h_dec, state_c_dec]

# Dense layer
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Function to generate predictions
def predict_sequence(encoder_model, decoder_model, input_seq, num_features, prediction_steps):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1 with only the start character
    target_seq = np.zeros((1, 1, num_features))

    # Populate the first feature of target sequence with the start character
    # target_seq[0, 0, feature_index] = value  # If you have a specific start value

    # Sampling loop for a batch of sequences
    decoded_sequence = []
    for _ in range(prediction_steps):
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Store the prediction
        decoded_sequence.append(output_tokens[0, 0, :])

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_features))
        target_seq[0, 0, :num_decoder_features] = output_tokens[0, 0, :]

        # Update states
        states_value = [h, c]

    return np.array(decoded_sequence)

# Example usage
input_seq = np.array([[[6,27,0,27.5,-79,25],[6,27,6,28.5,-79,25],[6,27,6,29.5,-79,25],[6,27,6,30.5,-79,25],[6,27,6,31.5,-78,25],[6,27,6,32.5,-77,25]]])  # Example input
print(type(input_seq))
prediction_steps = 5  # Number of future steps to predict
predicted_sequence = predict_sequence(encoder_model, decoder_model, input_seq, num_features, prediction_steps)
print(predicted_sequence)


<class 'numpy.ndarray'>
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 159ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
[[ 12.630088  -14.233355    6.0046463]
 [  6.4204288  -8.277024    3.6679015]
 [  6.2192583  -5.4994597   3.0940323]
 [  4.642462   -3.4300992   2.596946 ]
 [  3.2664518  -2.3475296   1.3877218]]


In [ ]:
# Inference model for predictions
encoder_model = Model(encoder_inputs, encoder_states) # Model() extracts features from the trained model

# decoder inference model, unlike training model, allows step-by-step sequence generation
decoder_state_input_h = Input(shape=(6,))
decoder_state_input_c = Input(shape=(6,))
# decoder states of previous timestep
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# decoder takes in the decoder input, as well as the previous state in initialization
# produces lstm output and updated states
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

# Function to generate predictions
def predict_sequence(encoder_model, decoder_model, input_seq, num_features, prediction_steps):
    states_value = encoder_model.predict(input_seq)

    target_seq = Input(shape=(None, num_features))


    decoded_sequence = []
    for _ in range(prediction_steps):
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        decoded_sequence.append(output_tokens[0, 0, :])
        target_seq = output_tokens[:, -1:, :]
        states_value = [h, c]

    return np.array(decoded_sequence)



In [ ]:
input_seq = np.array([[[6, 27, 0, 27.5, -79, 25], [6, 27, 0, 28.5, -79, 25]]])  # Shape (1, 1, 6)
prediction_steps = 10  # Number of points to predict
predictions = predict_sequence(encoder_model, decoder_model, input_seq, num_features, prediction_steps)
print(predictions)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/step


TypeError: int() argument must be a string, a bytes-like object or a real number, not 'NoneType'

In [ ]:
# visualizer